In [1]:
import os
import pandas as pd
import numpy as np
import scipy as sc
import scipy.misc
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont

In [2]:
# List of characters and classes
characterList = np.array('A B C D E F G H I J K L M N O P Q R S T U V W X Y Z'.split(' '))
numbersList = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype = int)
classes = np.arange(0, 38)
func = np.vectorize(str)
classesText = np.concatenate([func(numbersList).reshape(-1, 1), characterList.reshape(-1, 1)])

In [3]:
# function for selecting random character
def selectorOFCharacters (listOfCharactor):
    character = np.random.choice(listOfCharactor)
    if type(character) == np.str_:
        return str(character)
    else:
        return int(character)

In [4]:
# Generating fake number Plates characters for images
def plateGenerator (numberOfPlates):
    platesList = []
    for i in range(numberOfPlates):
        plateList = []
        for j in range(8):
            if j == 2:
                plateList.append(selectorOFCharacters(characterList))
            else:
                plateList.append(selectorOFCharacters(numbersList))
        platesList.append(plateList)
    platesList = np.array(platesList)
    return platesList

In [34]:
# function for setup repetitive setup images
def setupPNGCharacter (coordinates, character, width = 90, height = 75, font = "arial.ttf", fontSize = 95,
                      destination = 'E:/Hamed/Projects/Python/License Plate Detection/License-Plate-Detection/Characterslist/{}.png'):
    font = ImageFont.truetype(font, fontSize)
    img = Image.new('RGB', (width, height), color = (255, 255, 255))
    draw = ImageDraw.Draw(img)
    if type(coordinates) != tuple:
        raise TypeError('Coordinates must be tuple')
    draw = draw.text(coordinates, character, fill = (0, 0, 0), font = font)
    img.save(destination.format(character))

In [32]:
# Create PNG files of characters
def createPNGPieces (charactersList):
    if isinstance(charactersList[0], str):
        for character in charactersList:
            if character == 'C':
                setupPNGCharacter(coordinates = (10, -16), character = character)
            elif character == 'G':
                setupPNGCharacter(coordinates = (8, -15), character = character)
            elif character == 'I':
                setupPNGCharacter(coordinates = (32, -15), character = character)
            elif character == 'J':
                setupPNGCharacter(coordinates = (23, -15), character = character)
            elif character == 'L':
                setupPNGCharacter(coordinates = (18, -15), character = character)
            elif character == 'M':
                setupPNGCharacter(coordinates = (6, -15), character = character)
            elif character == 'O':
                setupPNGCharacter(coordinates = (8, -15), character = character)
            elif character == 'Q':
                setupPNGCharacter(coordinates = (7, -15), character = character, height = 78)
            elif character == 'W':
                setupPNGCharacter(coordinates = (0, -16), character = character)
            else:
                setupPNGCharacter(coordinates = (14, -14), character = character)
    else:
        for character in charactersList:
            character = str(character)
            setupPNGCharacter(coordinates = (-1, -15), character = character, width = 50, height = 75)
            setupPNGCharacter(coordinates = (-1, -15), character = character, width = 45, height = 65, fontSize = 85,
            destination = 'E:/Hamed/Projects/Python/License Plate Detection/License-Plate-Detection/Characterslist/smaller{}.png')

In [33]:
createPNGPieces(characterList)
createPNGPieces(numbersList)

In [26]:
# Preparing characters of number Plates for png files
def joiningPartsOfPlate (plate):
    return list(map(lambda number:''.join(number), plate))

In [29]:
# creating real number Plates
def createNumberPlates (plates):
    for plateNumber in range(len(plates)):
        image = Image.open('Plate Pieces/composite.png')
        background = image.copy()
        background.save('Plate Pieces/background.png')
        width = 85
        for i in range(8):
            if i < 2:
                background = Image.open('Plate Pieces/background.png')
                foreground = Image.open('Characterslist/{}.png'.format(plates[plateNumber, i]))
                background.paste(foreground, (width, 26))
                background.save('Plate Pieces/background.png')
                width += 55
            elif i == 2:
                background = Image.open('Plate Pieces/background.png')
                foreground = Image.open('Characterslist/{}.png'.format(plates[plateNumber, i]))
                background.paste(foreground, (width, 26))
                background.save('Plate Pieces/background.png')
                width += 95
            elif i > 2 and i < 6:
                background = Image.open('Plate Pieces/background.png')
                foreground = Image.open('Characterslist/{}.png'.format(plates[plateNumber, i]))
                background.paste(foreground, (width, 26))
                background.save('Plate Pieces/background.png')
                width += 55
            elif i == 6:
                background = Image.open('Plate Pieces/background.png')
                foreground = Image.open('Characterslist/smaller{}.png'.format(plates[plateNumber, i]))
                background.paste(foreground, (472, 42))
                background.save('Plate Pieces/background.png')
            else:
                background = Image.open('Plate Pieces/background.png')
                foreground = Image.open('Characterslist/smaller{}.png'.format(plates[plateNumber, i]))
                background.paste(foreground, (536, 42))
                background.save('Plate Pieces/background.png')
        background = Image.open('Plate Pieces/background.png')
        name = joiningPartsOfPlate([plates[plateNumber]])[0]
        background.save('Plate Pieces/{}.png'.format(name))
    os.remove(path = 'Plate Pieces/background.png')